# Parallel Inference

Demonstrates using joblib for parallel inferences.

## Huang graph

In [1]:
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.pptc.inferencecontroller import InferenceController
from random import choice, sample, randint

def do_inference(junction_tree):
    val = choice(['on', 'off'])
    ev = EvidenceBuilder() \
        .with_node(join_tree.get_bbn_node_by_name('a')) \
        .with_evidence(val, 1.0) \
        .build()
    
    join_tree.unobserve_all()
    join_tree.set_observation(ev)
    
    posteriors = join_tree.get_posteriors()
    return posteriors['h']

def do_random_inference(junction_tree):
    def get_ev(n, v):
        return EvidenceBuilder() \
            .with_node(join_tree.get_bbn_node_by_name(n)) \
            .with_evidence(v, 1.0) \
            .build()
    
    max_idx = randint(1, 3)
    names = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
    names = sample(names, len(names))[:max_idx]
    vals = [choice(['on', 'off']) for _ in range(len(names))]
    evidences = [get_ev(n, v) for n, v in zip(names, vals)]
    
    join_tree.unobserve_all()
    join_tree.update_evidences(evidences)
    
    posteriors = join_tree.get_posteriors()
    return posteriors['h']

# create the nodes
a = BbnNode(Variable(0, 'a', ['on', 'off']), [0.5, 0.5])
b = BbnNode(Variable(1, 'b', ['on', 'off']), [0.5, 0.5, 0.4, 0.6])
c = BbnNode(Variable(2, 'c', ['on', 'off']), [0.7, 0.3, 0.2, 0.8])
d = BbnNode(Variable(3, 'd', ['on', 'off']), [0.9, 0.1, 0.5, 0.5])
e = BbnNode(Variable(4, 'e', ['on', 'off']), [0.3, 0.7, 0.6, 0.4])
f = BbnNode(Variable(5, 'f', ['on', 'off']), [0.01, 0.99, 0.01, 0.99, 0.01, 0.99, 0.99, 0.01])
g = BbnNode(Variable(6, 'g', ['on', 'off']), [0.8, 0.2, 0.1, 0.9])
h = BbnNode(Variable(7, 'h', ['on', 'off']), [0.05, 0.95, 0.95, 0.05, 0.95, 0.05, 0.95, 0.05])

# create the network structure
bbn = Bbn() \
    .add_node(a) \
    .add_node(b) \
    .add_node(c) \
    .add_node(d) \
    .add_node(e) \
    .add_node(f) \
    .add_node(g) \
    .add_node(h) \
    .add_edge(Edge(a, b, EdgeType.DIRECTED)) \
    .add_edge(Edge(a, c, EdgeType.DIRECTED)) \
    .add_edge(Edge(b, d, EdgeType.DIRECTED)) \
    .add_edge(Edge(c, e, EdgeType.DIRECTED)) \
    .add_edge(Edge(d, f, EdgeType.DIRECTED)) \
    .add_edge(Edge(e, f, EdgeType.DIRECTED)) \
    .add_edge(Edge(c, g, EdgeType.DIRECTED)) \
    .add_edge(Edge(e, h, EdgeType.DIRECTED)) \
    .add_edge(Edge(g, h, EdgeType.DIRECTED))

join_tree = InferenceController.apply(bbn)

## Inference using list comprehension

In [2]:
import pandas as pd

pd.DataFrame([do_inference(join_tree) for _ in range(10)])

,on,off
0,0.7826,0.2174
1,0.7826,0.2174
2,0.8636,0.1364
3,0.8636,0.1364
4,0.8636,0.1364
5,0.7826,0.2174
6,0.7826,0.2174
7,0.8636,0.1364
8,0.7826,0.2174
9,0.8636,0.1364


In [3]:
pd.DataFrame([do_random_inference(join_tree) for _ in range(10)])

,on,off
0,0.942802,0.057198
1,0.896000,0.104000
2,0.641253,0.358747
3,0.647545,0.352455
4,0.782600,0.217400
5,0.950000,0.050000
6,0.734000,0.266000
7,0.896000,0.104000
8,0.821909,0.178091
9,0.950000,0.050000


## Inference using joblib

Can the join tree be serialized/deserialzed properly for use with joblib?

In [4]:
from joblib import Parallel, delayed

Parallel(n_jobs=2)(delayed(do_inference)(join_tree) for _ in range(10))

[{'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174},
 {'on': 0.7826, 'off': 0.2174}]

In [5]:
Parallel(n_jobs=2)(delayed(do_random_inference)(join_tree) for _ in range(10))

[{'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.8477732309663355, 'off': 0.15222676903366444},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.8477732309663355, 'off': 0.15222676903366444},
 {'on': 0.7340000000000001, 'off': 0.266},
 {'on': 0.7340000000000001, 'off': 0.266}]